**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

import random
from keras.callbacks import ModelCheckpoint
from keras.utils.np_utils import to_categorical
from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd,adam
from keras.layers import Convolution2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten

//anaconda/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [2]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act returns the next action that we choose accoding to the policy. It adopts the greedy action with the probability (1-epsilon), and takes random action with the probability epsilon. 

epsilon is important because this algorithm uses this parameter to control the balance between exploration and exploitation. Big value of epsilon means we consider more about exploration when we act.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [3]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [4]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array position records the position of the agent. The array board shows the reward for each position on the board.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [5]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, 4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
    # restart to a fresh game and get the initial state
        state = env.reset()
        game_over = False

        win = 0
        lose = 0

        while not game_over:
        # The agent performs an action
            action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

        # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
       
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 8.0/14.0. Average score (-6.0)
Win/lose count 10.0/11.0. Average score (-3.5)
Win/lose count 7.5/9.0. Average score (-2.8333333333333335)
Win/lose count 9.0/10.0. Average score (-2.375)
Win/lose count 13.0/14.0. Average score (-2.1)
Win/lose count 7.5/10.0. Average score (-2.1666666666666665)
Win/lose count 9.5/8.0. Average score (-1.6428571428571428)
Win/lose count 8.0/11.0. Average score (-1.8125)
Win/lose count 9.0/7.0. Average score (-1.3888888888888888)
Win/lose count 9.5/13.0. Average score (-1.6)
Final score: -1.6


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}





__Proof:__   
__(1)__  
If the sequence of rewards received after time step t is denoted $R_{t+1},R_{t+2},R_{t+3}, ...$. The agent tries to select actions so that the sum of the discounted rewards it receives over the future is maximized. Suppose it chooses $A_t$ to maximize the expected discounted return:

\begin{equation*}
G_{t}=R_{t+1}+ \gamma R_{t+2}+ \gamma^2 R_{t+3} +... = \sum^\infty_{k=0} \gamma^k R_{t+k+1}
\end{equation*}
where $ \gamma $ is a parameter,$0 \leq \gamma \leq 1$, called the discount rate

we deﬁne the value of a state s under a policy $ \pi $, denoted $ V_\pi(s, a) $, is the expected return when starting in s and following policy $ \pi $:

\begin{equation*}
V_\pi(s)= E_{\pi}[G_t｜S_t=s] =E_{\pi}[\sum^\infty_{k=0} \gamma^k R_{t+k+1} | S_t=s]
\end{equation*}

the value of taking action a in state s under a policy $ \pi $,  denoted  $ Q_\pi(s, a) $, as the expected return starting from s, taking the action a, and following policy $ \pi $:

\begin{align*}
Q_\pi(s,a)&= E_{\pi}[G_t｜S_t=s,A_t=a] \\
&= E_{\pi}[\sum^\infty_{k=0} \gamma^k R_{t+k+1} | S_t=s,A_t=a]\\
&= E_{\pi}[ R_{t+1} +\sum^\infty_{k=1} \gamma^k R_{t+k+1} | S_t=s,A_t=a]\\
&= E_{\pi}[ r(s,a) +\gamma E_{\pi}[G_t｜S_{t+1}=s',A_{t+1}=a']]\\
&= E_{\pi}[ r(s,a) +\gamma Q_\pi(s',a')]
\end{align*}



__(2)__  
For the optimal policy,  
\begin{align*}
V^*(s) &= max_\pi V_\pi(s)\\
Q^*(s,a) &= Q_\pi(s,a)
\end{align*}

Then we can write Q^*(s,a) as follows:  
\begin{equation*}
Q^*(s,a) = E_{\pi}[ r(s,a) + V^*(s_{t+1})| S_t=s,A_t=a])
\end{equation*}

The Bellman optimality equation denotes that the value of a state under an optimal policy must equal the expected return for the best action from that state:  
\begin{equation*}
V^*(s) = max_a Q^*(s,a)
\end{equation*}
So we get  
\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

__(3)__  
In deep Q learning, we take $ r(s,a)+\gamma\max_{a'}Q^{*}(s',a') $ as the target, and use "mse" loss, so the objective loss function can be written as:  
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [6]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)<self.max_memory:
            self.memory.append(m)
        else:
            self.memory.pop(0)
            self.memory.append(m)

    def random_access(self):
        return random.choice(self.memory)

***
The pipeline we will use for training is given below:

In [7]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [10]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        state = s.reshape(-1,5,5,self.n_state)
        actions_value = self.model.predict(state)
        
        action = np.argmax(actions_value[0])
        return action

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            random_mem = self.memory.random_access()
            state = random_mem[1].reshape(-1,5,5,self.n_state)
            values_tp = self.model.predict(state)
            input_states[i] = random_mem[0]
            target_q[i] = values_tp[0]
            target_q[i][random_mem[2]] = random_mem[3] + (1 - random_mem[4])*self.discount*np.max(values_tp[0])
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state)))
        model.add(Dense(512, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(4, activation='linear'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [11]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/020 | Loss 0.0049 | Win/lose count 2.5/4.0 (-1.5)
Epoch 001/020 | Loss 0.0091 | Win/lose count 4.0/3.0 (1.0)
Epoch 002/020 | Loss 0.0062 | Win/lose count 5.5/8.0 (-2.5)
Epoch 003/020 | Loss 0.0688 | Win/lose count 3.5/1.0 (2.5)
Epoch 004/020 | Loss 0.0591 | Win/lose count 5.5/3.0 (2.5)
Epoch 005/020 | Loss 0.0446 | Win/lose count 4.5/4.0 (0.5)
Epoch 006/020 | Loss 0.0068 | Win/lose count 3.5/1.0 (2.5)
Epoch 007/020 | Loss 0.0324 | Win/lose count 2.0/1.0 (1.0)
Epoch 008/020 | Loss 0.0067 | Win/lose count 3.5/6.0 (-2.5)
Epoch 009/020 | Loss 0.0041 | Win/lose count 4.5/1.0 (3.5)
Epoch 010/020 | Loss 0.0042 | Win/lose count 3.0/0 (3.0)
Epoch 011/020 | Loss 0.0476 | Win/lose count 6.5/2.0 (4.5)
Epoch 012/020 | Loss 0.0100 | Win/lose count 6.0/5.0 (1.0)
Epoch 013/020 | Loss 0.0085 | Win/lose count 5.0/0 (5.0)
Epoch 014/020 | Loss 0.0031 | Win/lose count 6.0/4.0 (2.0)
Epoch 015/020 | Loss 0.0102 | Win/lose count 4.0/4.0 (0.0)
Epoch 016/020 | Loss 0.0107 | Win/lose count 4.0/3.0 (1.0

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [81]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Convolution2D(filters=32,nb_row=3,nb_col=3, activation='relu',padding='same',input_shape=(5,5,self.n_state)))
        model.add(Convolution2D(filters=64,nb_row=3,nb_col=3, activation='relu',padding='same',input_shape=(5,5,self.n_state)))
        
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dense(4, activation='linear'))
        
        #print(model.summary())
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [59]:
epochs_train = 50
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train40.mp4'))

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(3, 3), activation="relu", input_shape=(5, 5, 2), padding="same", filters=32)`
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(3, 3), activation="relu", input_shape=(5, 5, 2), padding="same", filters=64)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_44 (Conv2D)           (None, 5, 5, 32)          608       
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 5, 5, 64)          18496     
_________________________________________________________________
flatten_26 (Flatten)         (None, 1600)              0         
_________________________________________________________________
dense_55 (Dense)             (None, 128)               204928    
_________________________________________________________________
dense_56 (Dense)             (None, 4)                 516       
Total params: 224,548
Trainable params: 224,548
Non-trainable params: 0
_________________________________________________________________
None
Epoch 000/050 | Loss 0.0037 | Win/lose count 7.5/5.0 (2.5)
Epoch 001/050 | Loss 0.0049 | Win/lose count 10.5/8.0 (2.5)
Epoch 002/05

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

Issue: With both algorithms, the player tends to be stuck somewhere and does not show obvious trend to explore.

With bigger value of temperature,  there are more 'postive reward' position and 'negative reward' positions in the grid, and this issue seems to be alleviated.

In [21]:
#epochs_train = 20
env = Environment(grid_size=size, max_time=T,temperature=0.5)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(3, 3), activation="relu", input_shape=(5, 5, 2), padding="same", filters=32)`
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(3, 3), activation="relu", input_shape=(5, 5, 2), padding="same", filters=64)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 5, 5, 32)          608       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 5, 5, 64)          18496     
_________________________________________________________________
flatten_7 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               204928    
_________________________________________________________________
dense_16 (Dense)             (None, 4)                 516       
Total params: 224,548
Trainable params: 224,548
Non-trainable params: 0
_________________________________________________________________
None
Test of the CNN
Win/lose count 13.5/3.0. Average score (10.5)
Win/lose count 12.5/5.0. Average score (9.0)
Win/lose count 7.0/2.0. 

In [22]:
HTML(display_videos('cnn_test9.mp4'))

In [0]:
HTML(display_videos('fc_test9.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [29]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            agent.set_epsilon(0.95*agent.epsilon)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, reward_game, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward_game
            if reward < 0:
                lose = lose -reward_game

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))
            agent.set_epsilon(np.fmax(0.05,agent.epsilon/2.))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] += 0.01

        reward = reward + self.board[self.x, self.y]
        reward_game = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, reward_game, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        #self.malus_position[self.x, self.y] = 0.1
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


In [101]:
# Training
epochs_train = 20
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.6, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore20.mp4'))

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(padding="same", filters=32, kernel_size=(3, 3), input_shape=(5, 5, 3), activation="relu")`
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(padding="same", filters=64, kernel_size=(3, 3), input_shape=(5, 5, 3), activation="relu")`


Epoch 000/020 | Loss 0.0086 | Win/lose count 6.5/9.0 (-2.5)
Epoch 001/020 | Loss 0.0060 | Win/lose count 5.5/4.0 (1.5)
Epoch 002/020 | Loss 0.0068 | Win/lose count 4.0/2.0 (2.0)
Epoch 003/020 | Loss 0.0205 | Win/lose count 6.0/5.0 (1.0)
Epoch 004/020 | Loss 0.0208 | Win/lose count 8.5/6.0 (2.5)
Epoch 005/020 | Loss 0.0584 | Win/lose count 6.5/4.0 (2.5)
Epoch 006/020 | Loss 0.0105 | Win/lose count 3.0/6.0 (-3.0)
Epoch 007/020 | Loss 0.0104 | Win/lose count 6.5/8.0 (-1.5)
Epoch 008/020 | Loss 0.0107 | Win/lose count 6.0/0.5 (5.5)
Epoch 009/020 | Loss 0.0173 | Win/lose count 10.0/4.0 (6.0)
Epoch 010/020 | Loss 0.0103 | Win/lose count 12.0/3.0 (9.0)
Epoch 011/020 | Loss 0.0154 | Win/lose count 10.0/4.5 (5.5)
Epoch 012/020 | Loss 0.0135 | Win/lose count 14.5/9.0 (5.5)
Epoch 013/020 | Loss 0.0277 | Win/lose count 14.5/4.0 (10.5)
Epoch 014/020 | Loss 0.0182 | Win/lose count 2.5/1.0 (1.5)
Epoch 015/020 | Loss 0.0286 | Win/lose count 7.5/2.0 (5.5)
Epoch 016/020 | Loss 0.0192 | Win/lose count 11

In [99]:
def test_explore(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
    # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
    # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
        # The agent performs an action
            action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
            prev_state = state
            state, reward,reward_game, game_over = env.act(action,train=False)

        # Update the counters
            if reward > 0:
                win = win + reward_game
            if reward < 0:
                lose = lose -reward_game

        # Apply the reinforcement strategy
            #loss = agent.reinforce(prev_state, state,  action, reward, game_over)
       
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [102]:
# Evaluation
epochs_test = 11
test_explore(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))


Win/lose count 10.5/3.0. Average score (7.5)
Win/lose count 2.5/1.0. Average score (4.5)
Win/lose count 5.5/0. Average score (4.833333333333333)
Win/lose count 2.5/0. Average score (4.25)
Win/lose count 2.5/2.0. Average score (3.5)
Win/lose count 10.5/1.0. Average score (4.5)
Win/lose count 7.5/0. Average score (4.928571428571429)
Win/lose count 8.5/1.0. Average score (5.25)
Win/lose count 3.5/2.0. Average score (4.833333333333333)
Win/lose count 4.0/2.0. Average score (4.55)
Win/lose count 9.5/0. Average score (5.0)
Final score: 5.0


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

By trainning a model based on the historical data of states and actions of the winning games (x = states, y = actions), we found that it can produce a comparable agent to the previous DQN_CNN model, with far less time.

The preselected data (data of winning games) is very helpful for the training of the agent. The agent will find the 'proper' action far quickly by mimicking the behavor of a winned agent.

In [82]:
# geberate winning games
def gen_wingame(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
    win_states = []
    
    win_actions = []
        
    for e in range(epochs):
    # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
    # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
        # The agent performs an action
            action = agent.act(state)
            agent.set_epsilon(0.95*agent.epsilon)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
            prev_state = state
            state, reward, reward_game, game_over = env.act(action, train=True)

        # Update the counters
            if reward > 0:
                win = win + reward_game
                win_states.append(prev_state)
                win_actions.append(action)
            if reward < 0:
                lose = lose - reward_game

        # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
       
        # Save as a mp4
        if e % 10 == 0:
            #env.draw(prefix+str(e))
            agent.set_epsilon(np.fmax(0.05,agent.epsilon/2.))

        # Update stats
        score = score + win-lose
    

        #print("Win/lose count {}/{}. Average score ({})"
        #      .format(win, lose, score/(1+e)))
    #print('Final score: '+str(score/epochs))
    return  win_states, win_actions

def gen_wingames(epochs,n_games,prefix=''):
    
    win_states = [] 
    win_actions = []
    for i in range(n_games):
        demon_env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
        demon_agent = DQN_CNN(size, lr=.1, epsilon = 0.5, memory_size=2000, batch_size = 32,n_state=3)
        win_states_i, win_actions_i = gen_wingame(demon_agent,demon_env,epochs,prefix=prefix)
        win_states.extend(win_states_i)
        win_actions.extend(win_actions_i)
        del demon_agent, demon_env
        print('------------{}th game-----------'.format(i))
    return win_states, win_actions

In [83]:
game_epochs = 25
n_games = 6
win_states_tp, win_actions_tp = gen_wingames(game_epochs,n_games,prefix='gen_wingames')
win_states.extend(win_states_tp)
win_actions.extend(win_actions_tp)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(padding="same", filters=32, kernel_size=(3, 3), input_shape=(5, 5, 3), activation="relu")`
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(padding="same", filters=64, kernel_size=(3, 3), input_shape=(5, 5, 3), activation="relu")`


------------0th game-----------
------------1th game-----------
------------2th game-----------
------------3th game-----------
------------4th game-----------
------------5th game-----------


***

In [97]:
#create a new agent learning from the learning games
class DQN_demonstrate(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_demonstrate, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Convolution2D(filters=32,nb_row=3,nb_col=3, activation='relu',padding='same',input_shape=(5,5,self.n_state)))
        model.add(Convolution2D(filters=64,nb_row=3,nb_col=3, activation='relu',padding='same',input_shape=(5,5,self.n_state)))
        
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dense(4, activation='softmax'))
        
        print(model.summary())
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), loss="categorical_crossentropy", metrics=['accuracy'])
        self.model = model
        
agent_demon = DQN_demonstrate(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 32,n_state=3)
save_model_path = '{}_weights.h5'.format('gen_wingames')
history = agent_demon.model.fit(np.array(win_states),
                to_categorical(win_actions, num_classes=4),
                epochs=100,
                validation_split=0.3,
                callbacks=[ModelCheckpoint(save_model_path, monitor='val_acc', save_best_only=True)
                ]
            )

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(padding="same", filters=32, kernel_size=(3, 3), input_shape=(5, 5, 3), activation="relu")`
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(padding="same", filters=64, kernel_size=(3, 3), input_shape=(5, 5, 3), activation="relu")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_61 (Conv2D)           (None, 5, 5, 32)          896       
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 5, 5, 64)          18496     
_________________________________________________________________
flatten_33 (Flatten)         (None, 1600)              0         
_________________________________________________________________
dense_69 (Dense)             (None, 128)               204928    
_________________________________________________________________
dense_70 (Dense)             (None, 4)                 516       
Total params: 224,836
Trainable params: 224,836
Non-trainable params: 0
_________________________________________________________________
None
Train on 3680 samples, validate on 1578 samples
Epoch 1/100
3680/3680 [==============================] - 2s 426us/step - loss: 0.79

In [100]:
epochs_test = 11
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
test_explore(agent_demon,env,epochs_test,prefix='demon_test_explore')
HTML(display_videos('demon_test_explore10.mp4'))

Win/lose count 10.5/3.0. Average score (7.5)
Win/lose count 16.0/12.0. Average score (5.75)
Win/lose count 18.0/9.0. Average score (6.833333333333333)
Win/lose count 17.5/9.0. Average score (7.25)
Win/lose count 18.5/15.0. Average score (6.5)
Win/lose count 7.0/3.0. Average score (6.083333333333333)
Win/lose count 19.5/11.0. Average score (6.428571428571429)
Win/lose count 11.5/8.0. Average score (6.0625)
Win/lose count 16.0/14.0. Average score (5.611111111111111)
Win/lose count 12.0/5.0. Average score (5.75)
Win/lose count 19.0/10.0. Average score (6.045454545454546)
Final score: 6.045454545454546
